# Доверительные интервалы для двух долей 

In [3]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины X∼N(μ,σ2) лежит в интервале μ±c⋅σ. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки.

In [6]:
round(scipy.stats.norm.ppf(1-0.003/2), 4)

2.9677

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [19]:
# создаем подвыборки принимавших аспирин и плацебо
aspirin = np.concatenate((np.ones(104),np.zeros(11037-104)))
placebo = np.concatenate((np.ones(189),np.zeros(11034-189)))

In [23]:
heart_attack_aspirin_prob = round(aspirin.mean(), 4)
heart_attack_placebo_prob = round(placebo.mean(), 4)
print heart_attack_placebo_prob - heart_attack_aspirin_prob  

0.0077


## Интервальные оценки долей

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [26]:
conf_interval_aspirin = proportion_confint(104, len(aspirin), method = 'wilson')
conf_interval_placebo = proportion_confint(189, len(placebo), method = 'wilson')

In [27]:
print 'interval for aspirin [%f, %f]' % conf_interval_aspirin
print 'interval for placebo [%f, %f]' % conf_interval_placebo

interval for aspirin [0.007783, 0.011404]
interval for placebo [0.014871, 0.019723]


Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

## Доверительный интервал для разности долей (независимые выборки)

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [31]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (round(left_boundary, 4), round(right_boundary, 4))

In [32]:
print "confidence interval: [%f, %f]" % proportions_confint_diff_ind(placebo, aspirin)

confidence interval: [0.004700, 0.010700]


Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин X∼Ber(p) часто вычисляют величину p/(1−p) которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо p нужно подставить p^. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как
(189/11034)/(1-189/11034) = 189/(11034-189) ≈ 0.0174

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [39]:
odds_with_aspirin = 104./(11037.-104)
odds_with_placebo = 189./(11034.-189)
print round(odds_with_placebo/odds_with_aspirin, 4) 

1.8321


Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
установите random seed=0;
сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [40]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [41]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [56]:
def odds(data):
    p = data.mean()
    return p / (1 - p)

np.random.seed(0)
odds_with_aspirin_data = np.array(map(odds, get_bootstrap_samples(aspirin, 1000)))
odds_with_placebo_data = np.array(map(odds, get_bootstrap_samples(placebo, 1000)))

In [63]:
print '95%% confidence interval for odds reletionship: ', stat_intervals(odds_with_placebo_data / odds_with_aspirin_data, 0.05)
      

95%% confidence interval for odds reletionship:  [1.44419465 2.34321168]
